# Missing Values

#### Installing required packages from requirement.txt

In [ ]:
pip install -r requirements.txt

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
TITANIC_TRAIN_DATA_URL = "https://stindatasamples.blob.core.windows.net/ai-ml-data/titanic/train.csv";
HOUSE_PRICES_TRAIN_DATA_URL = "https://stindatasamples.blob.core.windows.net/ai-ml-data/house-prices/train.csv";
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL)

In [ ]:
titanicDF.head()

## Missing values in dataset

1. Finding out all null values in the given data set for each column

In [ ]:
titanicDF.isnull().sum()

## Data Missing completely at random (MCAR) 

In [ ]:
titanicDF[titanicDF['Embarked'].isnull()]

Possiblely Embared data is missing completely at random as there is no sign of dependency 

## Missing data not at random

Checking percentage of missing value in cabin against survived column, as there might be interdependecy w.r.t. missing values in cabin column

In [ ]:
titanicDF['Cabin_null'] =np.where( titanicDF['Cabin'].isnull(),1,0)
titanicDF.groupby(['Survived'])['Cabin_null'].mean()

Cabin data has higher missing values where passenger didn't survive

## Techniques to handle Missing value 
1. Mean/Median/Mode
2. Random Sample Imputation
3. Capturing NaN values with new features
4. End of distribution imputation
5. Arbitrary Imputation 
6. Frequent categories imputation

### 1. Mean/Median/Mode

When - data missing completely at random (MCAR)

In [ ]:
def impute_nan_mean_median_mode(df, column, value):
    df[column + '_NonNull'] = df[column].fillna(value)

In [ ]:
impute_nan_mean_median_mode(titanicDF, 'Age', titanicDF['Age'].median())
titanicDF.isnull().mean()

In [ ]:
print(titanicDF['Age'].std())
print(titanicDF['Age_NonNull'].std())

In [ ]:
%matplotlib inline
fig = plt.figure()
ax = fig.add_subplot(111)
titanicDF['Age'].plot(kind='kde', ax=ax)
titanicDF['Age_NonNull'].plot(kind='kde', ax=ax, color='red')
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')


### 2. Random Sample Imputation

In [ ]:
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL, usecols=['Age', 'Fare', 'Survived'])


In [ ]:
titanicDF.isnull().mean()

In [ ]:
def impute_with_random_sample(df, column):
    df[column + '_random'] = df[column]
    random_samples = df[column].dropna().sample(df[column].isnull().sum(), random_state=0)  # randome_state = 0 means random values will not change
    random_samples.index=df[df['Age'].isnull()].index
    df.loc[df[column].isnull(), column+ "_random"] = random_samples


In [ ]:
impute_with_random_sample(titanicDF,'Age')
titanicDF.head()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
titanicDF['Age'].plot(kind='kde', ax=ax)
titanicDF['Age_random'].plot(kind='kde',ax=ax, color='red')
lines, labels = ax.get_legend_handles_labels()
ax.legend(lines, labels, loc='best')

### 3. Capturing NaN values with new features

In [ ]:
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL,usecols=['Age', 'Survived'])

In [ ]:
def impute_with_newfeature(df, column):
    df[column + '_NonNull'] = np.where(df[column].isnull(), 1, 0)

In [ ]:
impute_with_newfeature(titanicDF, 'Age')
titanicDF['Age'].fillna(titanicDF['Age'].mean(), inplace=True)
titanicDF.head()

### 4. End of distribution imputation

In [ ]:
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL, usecols=['Age', 'Fare', 'Survived'])
titanicDF.Age.hist(bins=50)

In [ ]:
titanicDF.Age.std()*3 

In [ ]:
titanicDF.Age.plot.kde()

In [ ]:
sns.boxplot('Age', data=titanicDF)

In [ ]:
def impute_with_end_distribution(df, column):
    df[column + '_end_distribution'] = df[column].fillna(df[column].mean() + 3*df[column].std())

In [ ]:
impute_with_end_distribution(titanicDF,'Age')

In [ ]:
sns.boxplot('Age_end_distribution', data=titanicDF) # outliers are gone

In [ ]:
titanicDF['Age_end_distribution'].hist(bins=50) # resultant rigth skewed data

### 5. Arbitrary Imputation 

In [ ]:
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL, usecols = ["Age","Fare","Survived"])

In [ ]:
titanicDF.Age.hist(bins=50)

In [ ]:
def impute_with_arbitary(df, column, value):
    df[column + "_arbitary"] = df[column].fillna(value)

In [ ]:
impute_with_arbitary(titanicDF, 'Age', 100)

In [ ]:
titanicDF.Age_arbitary.hist(bins=50)

### 6. Frequent categories imputation

In [ ]:
houseDF = pd.read_csv(HOUSE_PRICES_TRAIN_DATA_URL, usecols=['BsmtQual','FireplaceQu', 'GarageType','SalePrice'])

In [ ]:
houseDF.isnull().mean().sort_values(ascending=True)

In [ ]:
houseDF.groupby(['BsmtQual'])['BsmtQual'].count().sort_values(ascending=False).plot.bar()

In [ ]:
houseDF.FireplaceQu.value_counts().plot.bar()

In [ ]:
houseDF.GarageType.value_counts().plot.bar()

In [ ]:
def impute_with_most_frequent_category(df, column):
    most_freq_cat = df[column].value_counts().index[0]
    df[column].fillna(most_freq_cat, inplace=True)
    # OR df[column].fillna(df[column].mode(), inplace=True)

In [ ]:
impute_with_most_frequent_category(houseDF, 'BsmtQual')
houseDF.isnull().mean().sort_values(ascending=True)

### Ordinal Number encoding

In [ ]:
import datetime
date_list = [datetime.datetime.today() - datetime.timedelta(x) for x in range(0,15)]
dateDF = pd.DataFrame(date_list)
dateDF.columns=['Days']
dict_Week = {"Monday": 1, "Tuesday": 2, "Wednesday":3, "Thursday": 4, "Friday":5,"Saturday": 6, "Sunday":7}
dateDF['Week Day'] = dateDF["Days"].dt.day_name()

In [ ]:
dateDF['Week Day']= dateDF['Week Day'].map(dict_Week)
dateDF

### Categorical values by frequency count imputaion

In [ ]:
adultDF=pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data', header=None, index_col=None)

In [ ]:
adultDF

In [ ]:
columns = [1,3,5,6,7,8,9,13]

In [ ]:
adultDF = adultDF[columns]

In [ ]:
adultDF.columns = ['Employement', 'Degree', 'Status', 'Designation', 'Family_job', 'Race', 'Sex', 'Country']

In [ ]:
for column in adultDF.columns[:]:
    print(column, ": " , len(adultDF[column].unique()), 'labels')

In [ ]:
country_dict = adultDF['Country'].value_counts().to_dict()
adultDF['Country'] = adultDF['Country'].map(country_dict)
adultDF.head()

### Target guided ordinal encoding

In [ ]:
titanicDF = pd.read_csv(TITANIC_TRAIN_DATA_URL, usecols=['Cabin', 'Survived'])

In [ ]:
titanicDF['Cabin'].fillna('Missing', inplace=True)
titanicDF['Cabin'].unique()
# Stripping category for simplecity
titanicDF['Cabin'] = titanicDF['Cabin'].astype(str).str[0]